In [35]:
#################################
# Simple-Instagram-Scraper v1.0.3
# Release: 10.11.2020
# GitHub: do-me
#################################

from selenium import webdriver
from bs4 import BeautifulSoup as bs
import time
import re
from urllib.request import urlopen
import json
from pandas.io.json import json_normalize
import pandas as pd, numpy as np
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import random 
import string
from time import gmtime, strftime
import os

option = webdriver.ChromeOptions()
chrome_prefs = {}
option.experimental_options["prefs"] = chrome_prefs
chrome_prefs["profile.default_content_settings"] = {"images": 2}
chrome_prefs["profile.managed_default_content_settings"] = {"images": 2}

# Parameters

#---------------------------------------------------------------

# CRUCIAL PARAMS

# directory to chromedriver or geckodriver, find chromedriver here: https://chromedriver.chromium.org/
browser = webdriver.Chrome("yourpath/chromedriver.exe", options=option)

# user credentials
username = "username"
userpassword = "userpassword"

# which page?
pagetoscrape = "https://www.instagram.com/explore/locations/118546/thessaloniki/" # either hashtag, location id or user account possible
# pagetoscrape = "https://www.instagram.com/explore/tags/berlin/"

#---------------------------------------------------------------

# OPTIONAL PARAMS

# maximum posts to scrape
maxiter = 10000

# filename for output csv
out_csv = "-".join(pagetoscrape.split("/")[-3:])[:-1] + ".csv" # i.e.'118546-thessaloniki' for locations or tags-berlin for hashtags

# quite crucial but subject to trial and error due to unknown Instagram blocking policy: breaks
# set a random break duration for every iteration after opening one post and before going to the next one
short_pauseduration_min = 2 # seconds 
short_pauseduration_max = 2.7 # seconds

# set a random break duration for longer breaks...
long_pauseduration_min = 4.8 # seconds 
long_pauseduration_max = 10.5 # seconds

# ...for the following random iterations (number of iterations = index of scraped posts)
pauselist = random.sample(range(10, 10000), 400) # between 10 and 10000 generate list of 500 values # randomList.sort() for sorting
pauselist.append([x + int(random.uniform(1,10)) for x in np.arange(20, 10000, 50).tolist()]) # just to make sure: add list with values every 51th to 60th iteration  

#---------------------------------------------------------------

# ADVANCED PARAMS

# uncomment for saving in a folder called "data" you created before manually
# if os.getcwd().split("\\")[-1] != "data": # if you changed working dir already for some reason
#  os.chdir(r"data") 

In [29]:
#### some functions #####

def delete_cache(first=False):
     
    browser.execute_script("window.open('');")
    time.sleep(2)
    
    browser.switch_to.window(browser.window_handles[-1])
    time.sleep(2)
    
    browser.get('chrome://settings/clearBrowserData')
    time.sleep(2)
    
    actions = ActionChains(browser) 
    actions.send_keys(Keys.TAB * 2) # send right combination
    actions.perform()
    time.sleep(1)
    
    actions = ActionChains(browser) 
    actions.send_keys(Keys.DOWN * 4) # send right combination
    actions.perform()
    time.sleep(1)
    
    actions = ActionChains(browser) 
    actions.send_keys(Keys.TAB * 2) # send right combination
    actions.perform()
    time.sleep(1)
    
    if first == True: # only first time, chrome saves input from first time
        actions = ActionChains(browser) 
        actions.send_keys(Keys.SPACE) # send right combination
        actions.perform()
        time.sleep(1)
    
    actions = ActionChains(browser) 
    actions.send_keys(Keys.TAB * 3 + Keys.ENTER) # confirm
    actions.perform()
    time.sleep(15) # wait some time to finish
    
    browser.close() # close this tab
    browser.switch_to.window(browser.window_handles[0]) # switch back

def brsel(obj): # browser select, convenience function
    global browser
    try:
        return browser.find_elements_by_css_selector(obj)
    except:
        return np.nan
    
def long_pauseduration():
    return random.uniform(long_pauseduration_min, long_pauseduration_max)

def short_pauseduration():
    return random.uniform(short_pauseduration_min, short_pauseduration_max)

def nextpost():
    brsel(".coreSpriteRightPaginationArrow")[0].click()

def previouspost():
    brsel(".coreSpriteLeftPaginationArrow")[0].click()

def gibberish():
    return ''.join(random.choice(string.ascii_lowercase) for _ in range(10))

def parsepost(): # parse function when post is displayed in gallery view, could also be executed manually
    global browser
    
    # values always existent
    p_time = brsel('time')[0].get_attribute('datetime')
    p_user_name = brsel('.ZIAjV')[0].text 
    p_user_id = brsel('.ZIAjV')[0].get_attribute('href').split("/")[-2] 
    p_post_id = browser.current_url.split("/")[-2]

    # values sometimes existent wrap in try except function
    p_loc_name = np.nan
    p_loc_slug = np.nan
    p_loc_id = np.nan
    p_likes = np.nan
    p_clicks = np.nan
    p_text = np.nan
    p_hashtags = np.nan
    p_alt = np.nan
    
    if len(brsel('.O4GlU'))>0: # if location exists
        p_loc_name = brsel('.O4GlU')[0].text
        p_loc_slug = brsel('.O4GlU')[0].get_attribute('href').split("/")[-2]
        p_loc_id = brsel('.O4GlU')[0].get_attribute('href').split("/")[-3]
        
    if len(brsel('.Nm9Fw button span'))>0: # likes
        p_likes = brsel('.Nm9Fw button span')[0].text # likes # can be non
           
    if len(brsel('.vcOH2'))>0: # clicks
        p_clicks = brsel('.vcOH2')[0].text  
        
    if len(brsel('ul li'))>0:
        p_text = brsel('ul li span')[1].text # full text including hashtags
        if len(brsel('ul li')[0].find_elements_by_css_selector('span')) > 0: # only hashtags
            if len(brsel('ul li span')[1].find_elements_by_css_selector('a'))>0:
                p_hashtags = [x.text for x in brsel('ul li span')[1].find_elements_by_css_selector('a')]  
    
    if len(brsel('._2dDPU .KL4Bh img'))>0:
        try: 
            p_alt = brsel('._2dDPU .KL4Bh img')[0].get_attribute('alt')
        except:
            p_alt = np.nan
    
    return [p_time,p_user_name,p_user_id,p_post_id,p_loc_name,p_loc_slug,p_loc_id,p_likes,p_clicks,p_text,p_hashtags,p_alt]


In [30]:
# login
browser.get('https://www.instagram.com/accounts/login/') # open login page and log in
time.sleep(3)
browser.find_elements_by_css_selector('.bIiDR')[0].click() # confirm cookies
time.sleep(1)

emailInput = browser.find_elements_by_css_selector('form input')[0]
passwordInput = browser.find_elements_by_css_selector('form input')[1]

emailInput.send_keys(username)
passwordInput.send_keys(userpassword)
passwordInput.send_keys(Keys.ENTER)

time.sleep(3)
browser.find_elements_by_css_selector('button')[1].click() # dont save user credentials (not possible)
time.sleep(3)

In [ ]:
# initialize page
browser.get(pagetoscrape)
time.sleep(2)
brsel('._9AhH0')[9].click()  # get first post ignoring top posts

# until here one could go manually correct

In [ ]:
posts = [] # emtpy list 
# execute this cell only once in the beginning 
# if for some reason the loop in the next cell stops, you can simple reexecute the next cell
# this list then contains the previously scraped information and adds new posts

In [ ]:
i = 1
while i < maxiter:
    print("Iteration: " + str(i), end="\r")
    
    # if not loading go back and forth
    # if fails again, set np.nan
    try: 
        currpost = parsepost()
    except IndexError:
        previouspost()
        time.sleep(3)
        nextpost()
        time.sleep(3)
        try: 
            currpost = parsepost()
        except IndexError:
            p_post_id = browser.current_url.split("/")[-2]
            posts.append([np.nan,np.nan,np.nan,p_post_id,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan])
            if i in pauselist:
                time.sleep(long_pauseduration())

            if len(brsel(".coreSpriteRightPaginationArrow"))==0: # break if come to the end or blocked entirely
                print("Ended at iteration:" +str(i))
                break

            nextpost()
            time.sleep(short_pauseduration())
            i += 1
            continue
    
    posts.append(currpost)
    
    if i in pauselist:
        time.sleep(long_pauseduration())
    
    if len(brsel(".coreSpriteRightPaginationArrow"))==0: # break if come to the end or blocked entirely
        print("Ended at iteration:" +str(i))
        break
        
    if i % 11 == 0: # every nth iteration
        try:
            brsel('form')[0].click()
            try:
                brsel('form textarea')[0].send_keys(gibberish())
            except IndexError:
                time.sleep(0.3)          
        except:
            time.sleep(0.2)
        time.sleep(1.2)
    
    if i == 100: # only first time cache settings
        delete_cache(first=True)
        nextpost()
        time.sleep(short_pauseduration())
        i += 1
        continue
        
    if i % 100 == 0: # every nth iteration
        delete_cache()
         
    nextpost()
    time.sleep(short_pauseduration())
    i += 1

In [ ]:
df = pd.DataFrame(posts)
df.columns = ["time","user_name","user_id","post_id","loc_name","loc_slug","loc_id","likes","clicks","text","hashtags","alt"]

# check if file already exists
if os.path.isfile(out_csv): # if exists append new posts to old file 
    tf = pd.read_csv(out_csv)
    tf = tf.append(df)
    tf.to_csv(out_csv,index=False)

# else create new file
else: 
    df.to_csv(out_csv,index=False)

In [ ]:
# display just a couple of rows
# df

# display all rows
# pd.set_option('display.max_rows', df.shape[0]+1) # display all rows
# df